In [9]:
import os
import numpy as np
import pandas as pd
from scipy.signal import resample
from datasets import Dataset
import wfdb  # For reading WFDB records
from datasets import Features, Array2D, Value

In [5]:
# Constants
BASE_PATH = "physionet.org/files/picsdb/1.0.0/"
OUTPUT_PATH = "HTR-2024-brachy"
TARGET_SAMPLING_RATE = 250  # Target sampling rate for all signals (Hz)
RESP_SAMPLING_RATE = 250  # Assuming all respiration signals will be resampled to 250Hz

# ECG Sampling Rates for specific infants
ECG_SAMPLING_RATES = {1: 250, 5: 250}  # ECG sampling rates for infants 1 and 5
DEFAULT_ECG_SAMPLING_RATE = 500  # Default ECG sampling rate

In [3]:
# Helper Function: Resample signals to target sampling rate
def resample_signal(signal, original_rate, target_rate):
    if original_rate != target_rate:
        num_samples = int(len(signal) * target_rate / original_rate)
        return resample(signal, num_samples)
    return signal

# Helper Function: Load WFDB signal and annotations
def load_signals_and_annotations(infant_id):
    try:
        # ECG
        ecg_record_name = f"infant{infant_id}_ecg"
        ecg_record = wfdb.rdrecord(os.path.join(BASE_PATH, ecg_record_name))
        ecg_sampling_rate = ECG_SAMPLING_RATES.get(infant_id, DEFAULT_ECG_SAMPLING_RATE)
        ecg_signal = ecg_record.p_signal[:, 0]  # Assuming single channel ECG
        ecg_signal = resample_signal(ecg_signal, ecg_sampling_rate, TARGET_SAMPLING_RATE)
        
        # Respiration
        resp_record_name = f"infant{infant_id}_resp"
        resp_record = wfdb.rdrecord(os.path.join(BASE_PATH, resp_record_name))
        resp_sampling_rate = resp_record.fs  # Original respiration sampling rate
        resp_signal = resp_record.p_signal[:, 0]  # Assuming single channel Respiration
        resp_signal = resample_signal(resp_signal, resp_sampling_rate, RESP_SAMPLING_RATE)
        
        # Load Bradycardia Annotations
        brady_ann = wfdb.rdann(os.path.join(BASE_PATH, ecg_record_name), 'atr')
        brady_times = np.array(brady_ann.sample) / ecg_sampling_rate  # Convert to seconds
        
        return ecg_signal, resp_signal, brady_times
    except Exception as e:
        print(f"Error loading data for infant {infant_id}: {e}")
        return None, None, None


In [7]:
# Process Dataset
data = []
infants = range(1, 11)  # Infants 1 to 10

for infant_id in infants:
    print(f"Processing Infant {infant_id}...")
    ecg_signal, resp_signal, brady_times = load_signals_and_annotations(infant_id)
    
    if ecg_signal is None or resp_signal is None:
        print(f"Skipping Infant {infant_id} due to loading error.")
        continue
    
    # Synchronize signals
    min_length = min(len(ecg_signal), len(resp_signal))
    ecg_signal = ecg_signal[:min_length]
    resp_signal = resp_signal[:min_length]
    
    # Combine ECG and Respiration into a 2-channel signal
    combined_signal = np.stack([ecg_signal, resp_signal], axis=0)  # Shape: (2, time)
    
    # Create Bradycardia Labels
    brady_labels = np.zeros(min_length, dtype=int)
    for brady_time in brady_times:
        brady_idx = int(brady_time * TARGET_SAMPLING_RATE)
        if brady_idx < len(brady_labels):
            brady_labels[brady_idx] = 1  # Mark bradycardia onset
    
    # Aggregate Bradycardia within 15s segments
    segment_length = TARGET_SAMPLING_RATE * 15  # 15 seconds
    total_segments = len(ecg_signal) // segment_length
    
    for seg in range(total_segments):
        start = seg * segment_length
        end = start + segment_length
        segment = combined_signal[:, start:end]
        label = 1 if np.any(brady_labels[start:end]) else 0
        data.append({
            "infant_id": infant_id,
            "segment_id": seg,
            "input": segment.T.tolist(),  # Shape: (3750, 2) -> transpose for channels last
            "label": label
        })
    
    print(f"Infant {infant_id}: {total_segments} segments processed.")

# Convert to Pandas DataFrame
df = pd.DataFrame(data)

# Verify DataFrame
print(f"Total segments: {len(df)}")
print(df.head())

Processing Infant 1...
Infant 1: 10947 segments processed.
Processing Infant 2...
Infant 2: 10521 segments processed.
Processing Infant 3...
Infant 3: 10491 segments processed.
Processing Infant 4...
Infant 4: 11227 segments processed.
Processing Infant 5...
Infant 5: 11700 segments processed.
Processing Infant 6...
Infant 6: 11665 segments processed.
Processing Infant 7...
Infant 7: 4880 segments processed.
Processing Infant 8...
Infant 8: 5904 segments processed.
Processing Infant 9...
Infant 9: 16875 segments processed.
Processing Infant 10...
Infant 10: 11344 segments processed.
Total segments: 105554
   infant_id  segment_id                                              input  \
0          1           0  [[-0.019983521088922047, 5.627044814449522], [...   
1          1           1  [[-0.07493820408345768, -0.5535211537849853], ...   
2          1           2  [[-0.054954682994535625, -0.546168427546254], ...   
3          1           3  [[-0.001248970068057628, -0.32019547734062365

In [10]:
# Define Custom Features Schema
features = Features({
    "infant_id": Value("int32"),
    "segment_id": Value("int32"),
    "input": Array2D(shape=(2, 3750), dtype="float32"),
    "label": Value("int64")
})

# Convert to Hugging Face Dataset with Custom Features
hf_dataset = Dataset.from_pandas(df, features=features)

# Save Dataset to Disk in Parquet Format
hf_dataset.save_to_disk(OUTPUT_PATH)

print(f"Dataset successfully saved to {OUTPUT_PATH}")

Saving the dataset (0/10 shards):   0%|          | 0/105554 [00:00<?, ? examples/s]

Dataset successfully saved to HTR-2024-brachy
